#Surivor - Outwit, Outplay, Outlast

Adam - Ian, Angie, Coby
V -  Tom, Janu, Katie

## Imports and functions

In [12]:
%pip install gender_guesser
%pip install pymysql

from bs4 import BeautifulSoup

import requests
import urllib.request
import time
import random
import requests
import random
import os
import pymysql
import time
import gender_guesser.detector as gender
import pandas as pd
import numpy as np

STATES = {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'D.C.': 'DC', 'Florida': 'FL', 
          'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 
          'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 
          'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 
          'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 
          'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA.', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


SEASONS = ['Borneo', 'The_Australian_Outback', 'Africa', 'Marquesas', 'Thailand', 'The_Amazon', 'Pearl_Islands', 'All-Stars', 'Vanuatu', 'Palau'] 

def getSeasonURL(season_num):
  wiki_url = 'https://en.wikipedia.org/wiki/Survivor:_'
  return wiki_url + SEASONS[ season_num - 1 ]

def getSurvivorId(first_name, last_name):
  first_name = "'"+first_name+"'"
  last_name = "'"+last_name.strip(" ")+"'"
  query = "Select survivor_id from survivor_data.Players where first_name ="+first_name+" and last_name="+last_name+";"
  cur.execute(query)
  id = cur.fetchone()[0]
  return str(id)

def executeQuery(query):
  cur.execute(query)

def commitQuery():
  conn.commit()

def findStateInString(s):
  word = ''
  for char in s:
    if word in states:
      return word
    else:
      word += char

def addQuotes(s):
  return " '"+s+"' "

def getGender(name):
  d = gender.Detector()
  g = d.get_gender(name).split("_")

  if len(g) == 2:
    gn = g[-1]
  else:
    gn = g[0]
    
  if gn == "male":
    return 'M'
  else:
    return 'F'

def getPlayerTableInfoRandom(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table', {'class': 'wikitable'})[0]
  table_rows = table.find_all('tr')

  rows = []
  for tr in table_rows:
      td = tr.find_all('td')
      td_row = [i.text for i in td]
      rows.append(td_row[:-1])
      random.shuffle(rows)

  return rows

def getPlayerTableInfo(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find_all('table', {'class': 'wikitable'})[0]
  table_rows = table.find_all('tr')

  rows = []
  for tr in table_rows:
      td = tr.find_all('td')
      td_row = [i.text for i in td]
      rows.append(td_row[:-1])

  return rows

## AWS Login

In [9]:
  succeess = False

  while not succeess:
    password = input("Password: ");
    try:
      conn = pymysql.connect(
          host='aws-survivor.cpapv35x7rsh.us-east-2.rds.amazonaws.com',
          port=int(3306),
          user="admin",
          passwd=password,
          db="survivor_data",
          charset='utf8mb4')
      cur = conn.cursor()
      succeess = True
    except:
      print("Wrong Password")
  print("Connection Made")

Password: Frederickwilliams121234
Connection Made


## Adding Players to Players Table

In [14]:
url = getSeasonURL(10)
rows =getPlayerTableInfoRandom(url)
print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0]
      last_name  = name_age[:-2].split(" ", 1)[1]
      age = name_age[-2:]
      gen = getGender(first_name)
      state = r_split[-1].strip("\n")[1:]

      print(first_name, last_name, age, gen, state)

      try:
        query = "INSERT INTO `survivor_data`.`Players` (`first_name`, `last_name`, `sex`, `state`) VALUES ("+addQuotes(first_name)+","+addQuotes(last_name)+","+addQuotes(gen)+","+addQuotes(STATES[state])+");"
        print(query)
        #executeQuery(query)
      except:
        print("Error")

#commitQuery()

[['Jonathan Libby23, Dallas, Texas\n', 'None[a]\n', '\n'], ['James Miller33, Mobile, Alabama\n', 'Ulong\n'], ['Janu Tornell39, Las Vegas, Nevada\n', 'Koror\n'], ['Gregg Carey27, Chicago, Illinois\n', 'Koror\n'], ['Willard Smith56, Bellevue, Washington\n', 'Koror\n'], ['Kimberly "Kim" Mullen25, Huber Heights, Ohio\n', 'Ulong\n'], ['Coby Archa32, Athens, Texas\n', 'Koror\n', 'Koror\n'], ['Stephenie LaGrossa24, Toms River, New Jersey\n', 'Ulong\n'], ['Ian Rosenberger23, Ambridge, Pennsylvania\n', 'Koror\n'], ['Caryn Groedel46, Solon, Ohio\n', 'Koror\n'], ['Jeff Wilson21, Ventura, California\n', 'Ulong\n'], [], ['Jolanda Jones38, Houston, Texas\n', 'Ulong\n'], ['Wanda Shirk55, Ulysses, Pennsylvania\n', 'None[a]\n'], ['Angie Jakusz †24, New Orleans, Louisiana\n', 'Ulong\n'], ['Ibrehem Rahman27, Birmingham, Alabama\n', 'Ulong\n'], ['Tom Westman40, Sayville, New York\n', 'Koror\n'], ['Jennifer "Jenn" Lyon †32, Encino, California\n', 'Koror\n'], ['Bobby Jon Drinkard27, Los Angeles, California\

## Add Players to Seasons Table

In [24]:
season = 10
url = getSeasonURL(season)
rows =getPlayerTableInfoRandom(url)
print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0]
      last_name  = name_age[:-2].split(" ", 1)[1]
      age = name_age[-2:]



      try:
        
        survivor_id = getSurvivorId(first_name, last_name)
        query = "INSERT INTO `survivor_data`.`Seasons` (season_id, survivor_id, age)"+" VALUES('"+str(season)+"','"+str(survivor_id)+"','"+str(age)+"');"
        print(query)
        #executeQuery(query)
      except:
        print(first_name, last_name, age)

#commitQuery()


[['Ashlee Ashby22, Easley, South Carolina\n', 'Ulong\n'], ['Wanda Shirk55, Ulysses, Pennsylvania\n', 'None[a]\n'], ['Janu Tornell39, Las Vegas, Nevada\n', 'Koror\n'], ['Bobby Jon Drinkard27, Los Angeles, California\n', 'Ulong\n'], ['Tom Westman40, Sayville, New York\n', 'Koror\n'], ['Angie Jakusz †24, New Orleans, Louisiana\n', 'Ulong\n'], ['Willard Smith56, Bellevue, Washington\n', 'Koror\n'], ['Ian Rosenberger23, Ambridge, Pennsylvania\n', 'Koror\n'], ['Jennifer "Jenn" Lyon †32, Encino, California\n', 'Koror\n'], ['Gregg Carey27, Chicago, Illinois\n', 'Koror\n'], ['Caryn Groedel46, Solon, Ohio\n', 'Koror\n'], ['Ibrehem Rahman27, Birmingham, Alabama\n', 'Ulong\n'], ['James Miller33, Mobile, Alabama\n', 'Ulong\n'], ['Stephenie LaGrossa24, Toms River, New Jersey\n', 'Ulong\n'], ['Katie Gallagher29, Merced, California\n', 'Koror\n'], ['Kimberly "Kim" Mullen25, Huber Heights, Ohio\n', 'Ulong\n'], ['Coby Archa32, Athens, Texas\n', 'Koror\n', 'Koror\n'], ['Jonathan Libby23, Dallas, Texas\n'

## Add Votes for in Season Table

In [ ]:
a = [ ['Lea', 'Masters'], ['Lisa', 'Keiffer'], ['John', 'Palyok'] ]
v = [ ['Brady', 'Finta'], ['Julie', 'Berry'], ['Chris', 'Daugherty'] ]
season = 9


for player in a:
  survivor_id = getSurvivorId(first_name=player[0], last_name=player[1])
  query = "UPDATE `survivor_data`.`Seasons` SET `vote_adam` = 1 WHERE `season_id` ='"+str(season)+"' and survivor_id ='"+survivor_id+"';"
  executeQuery(query)
  print(query)

for player in v:
  survivor_id = getSurvivorId(first_name=player[0], last_name=player[1])
  query = "UPDATE `survivor_data`.`Seasons` SET `vote_victoria` = 1 WHERE `season_id` ='"+str(season)+"' and `survivor_id ='"+survivor_id+"';"
  executeQuery(query)
  print(query)

commitQuery()

## Set Players Place in the Season

In [ ]:
season = 8
url = SEASONS[season - 1]
rows =getPlayerTableInfo(url)
rows = rows[1:]
place = len(rows)

print(rows)

for row in rows:
  if len(row) !=0:
    r_split = row[0].split(",")

    if len(r_split) == 3:
      name_age = r_split[0]
      first_name = name_age[:-2].split(" ", 1)[0].strip(" ")
      last_name  = name_age[:-2].split(" ", 1)[1].strip(" ").strip("'")
      #survivor_id = getSurvivorId(first_name, last_name)

      try:
        survivor_id = getSurvivorId(first_name, last_name)
        query = "UPDATE `survivor_data`.`Seasons` SET place ="+ str(place) +" WHERE `season_id` = 8 and `survivor_id` ='"+survivor_id+"';"
        #executeQuery(query)

      except:
        print("Error: ", first_name, last_name) 
        
  place -= 1   

#commitQuery()

## Getting Challenge Winners

In [ ]:
response = requests.get(getSeasonURL(7))
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find_all('table', {'class': 'wikitable'})[1]
table_rows = table.find_all('tr')

rows = []
for tr in table_rows:
    td = tr.find_all('td')
    td_row = [i.text for i in td]
    if len(td_row) != 0:
      rows.append(td_row)

print(rows)

count = 0

for epi in rows:
  if count >= 8:
    reward = 'N/A'
    immunity = 'N/A'

    print(epi)

    if epi[-1].strip("\n") != 'Runner-up' or epi[-1].strip("\n") != 'Sole Survivor':
      if epi[-1][:2] == '13' or epi[-1][:2] == '14':
        immunity = epi[-3].strip("\n")
      elif len(epi) >= 4:
        reward = epi[-4].strip("\n")
        immunity = epi[-3].strip("\n")
    
    print(reward, immunity)
  count +=1